In [2]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path

from models import PoseData, setup_logging
from posescript_generator import PoseScriptGenerator

In [3]:
def extract_player_keypoints(df_row, player='a', confidence_threshold=0.3):
    """Extract player keypoints from a CSV row using bounding box normalization.
    
    Args:
        df_row: DataFrame row containing pose data
        player: Player identifier ('a' or 'b')
        confidence_threshold: Minimum confidence score for keypoints (0.0 to 1.0)
        
    Returns:
        List of (x, y, confidence) tuples representing COCO keypoint coordinates
    """
    if player not in ['a', 'b']:
        raise ValueError("Player must be 'a' or 'b'")
    
    keypoint_names = [
        'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear',
        'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
        'left_wrist', 'right_wrist', 'left_hip', 'right_hip',
        'left_knee', 'right_knee', 'left_ankle', 'right_ankle'
    ]
    
    # Get bounding box from CSV - this provides proper normalization context
    bbox_prefix = f'{player}bb'  # 'abb' for player a, 'bbb' for player b
    x_min = df_row[f'{bbox_prefix}_xmin']
    y_min = df_row[f'{bbox_prefix}_ymin'] 
    x_max = df_row[f'{bbox_prefix}_xmax']
    y_max = df_row[f'{bbox_prefix}_ymax']
    
    # Calculate bounding box dimensions
    bbox_width = x_max - x_min
    bbox_height = y_max - y_min
    bbox_center_x = (x_min + x_max) / 2.0
    bbox_center_y = (y_min + y_max) / 2.0
    
    keypoints = []
    
    for kp_name in keypoint_names:
        x_col = f'{player}_{kp_name}_x'
        y_col = f'{player}_{kp_name}_y'
        conf_col = f'{player}_{kp_name}_confidence'
        
        x_img = df_row[x_col] 
        y_img = df_row[y_col] 
        y_img = bbox_height - y_img
        conf = df_row[conf_col]
        
        # Return 3D format for PoseScript: (z, y, x) where z=0 for 2D poses
        keypoints.append((x_img, y_img, conf))
    
    return keypoints

In [ ]:
# Read the CSV file
csv_path = "/Users/chanakyd/work/badminton/VB_DATA/poses/14_Smash/2022-08-30_18-00-09_dataset_set1_087_006675_006699_B_14.csv"
player = 'b'

df = pd.read_csv(csv_path)

# Initialize PoseScriptGenerator
pose_script_gen = PoseScriptGenerator()
if not pose_script_gen.initialize():
    print("❌ Failed to initialize PoseScriptGenerator")

# Generate textual description for player 'b'
descriptions = []
for _, row in df.iterrows():
    keypoints = extract_player_keypoints(row, player=player)
    pose_data = PoseData(keypoints)
    description = pose_script_gen.generate_description(pose_data)
    descriptions.append(description)

# Print the generated descriptions
for desc in descriptions:
    print(desc)

INFO:posescript_generator:PoseScriptGenerator initialized with model: /Users/chanakyd/work/badminton/posescript/capgen/seed1/checkpoint_best.pth


SyntaxError: 'return' outside function (2277288559.py, line 11)